# EV Battery Health Monitor - User Profile V2 Development

This notebook is for developing and testing user_profiles_v2.py with realistic charging behavior patterns based on research data.

In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Add backend to path
sys.path.append('../backend')

from simulation.user_profiles_v2 import UserProfile, UserBehaviorSimulator

## Test COMMON_DRIVER Profile (Typical EV Driver)

The COMMON_DRIVER profile represents typical EV driver behavior:
- SoC range: 25-85% (research-based common case)
- Charging frequency: 4-5 times per week (realistic average)
- Daily distance: 45-65km (center of research range)
- Mix of night and opportunity charging

In [ ]:
# Test COMMON_DRIVER behavior over a week
sim = UserBehaviorSimulator(UserProfile.COMMON_DRIVER, seed=42)

# Simulate a week
current_soc = 80.0  # Start with typical charge
data = []

for day in range(7):
    day_name = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"][day]
    is_weekend = day >= 5
    
    # Daily distance
    daily_distance = sim.get_daily_distance(is_weekend)
    
    # Energy consumption (simplified: ~4% SoC per 20km)
    energy_consumption_pct = (daily_distance / 20.0) * 4.0
    soc_after_driving = max(0, current_soc - energy_consumption_pct)
    
    # Check charging decision (evening)
    should_charge, target_soc = sim.should_charge(soc_after_driving, hour=22, day_of_week=day)
    
    final_soc = target_soc if should_charge else soc_after_driving
    
    data.append({
        'Day': day_name,
        'SoC_Start': current_soc,
        'Distance_km': daily_distance,
        'SoC_After_Drive': soc_after_driving,
        'Charged': should_charge,
        'Target_SoC': target_soc if should_charge else None,
        'SoC_End': final_soc
    })
    
    current_soc = final_soc

df = pd.DataFrame(data)
print("COMMON_DRIVER Weekly Behavior:")
print(df.to_string(index=False))

In [ ]:
# Analyze results
total_distance = df['Distance_km'].sum()
avg_daily_distance = df['Distance_km'].mean()
charges_this_week = df['Charged'].sum()
final_soc = df['SoC_End'].iloc[-1]

print(f"\n=== Weekly Analysis ===")
print(f"Total distance: {total_distance:.1f}km")
print(f"Average daily: {avg_daily_distance:.1f}km")
print(f"Charging sessions: {charges_this_week}")
print(f"Final SoC: {final_soc:.1f}%")

# Validation against research
print(f"\n=== Research Validation ===")
print(f"Daily distance in range (30-80km): {'✅' if 30 <= avg_daily_distance <= 80 else '❌'} ({avg_daily_distance:.1f}km)")
print(f"Charging frequency in range (3-7/week): {'✅' if 3 <= charges_this_week <= 7 else '❌'} ({charges_this_week}/week)")
print(f"Safe SoC management (>20%): {'✅' if final_soc >= 20 else '❌'} ({final_soc:.1f}%)")

In [ ]:
# Visualize SoC patterns
plt.figure(figsize=(12, 6))

days = range(len(df))
plt.plot(days, df['SoC_Start'], 'o-', label='SoC Start of Day', color='blue')
plt.plot(days, df['SoC_After_Drive'], 's-', label='SoC After Driving', color='orange')
plt.plot(days, df['SoC_End'], '^-', label='SoC End of Day', color='green')

# Highlight charging events
charging_days = df[df['Charged']].index
for day in charging_days:
    plt.axvspan(day-0.1, day+0.1, alpha=0.3, color='yellow', label='Charged' if day == charging_days[0] else "")

# Add reference lines for optimal SoC range
plt.axhline(y=85, color='red', linestyle='--', alpha=0.7, label='Target Max (85%)')
plt.axhline(y=25, color='red', linestyle='--', alpha=0.7, label='Target Min (25%)')

plt.xlabel('Day of Week')
plt.ylabel('State of Charge (%)')
plt.title('COMMON_DRIVER SoC Management Pattern (Typical EV Driver)')
plt.xticks(days, df['Day'])
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim(0, 100)
plt.tight_layout()
plt.show()

## Next: Add More Profiles

We can iteratively add more user profiles here:
- COMMUTER (high daily distance, regular patterns)
- WEEKEND_WARRIOR (low weekday, high weekend)
- SPONTANEOUS (higher variance, less planning)
- etc.

Each will be calibrated against the research data for realistic behavior.